In [1]:
import simplekml
import glob
import os
import pykml
from pykml import parser
from pykml.factory import KML_ElementMaker as KML
from lxml import etree, objectify

In [4]:
base_dir = '/home/student.unimelb.edu.au/shorte1/Documents/ACF_consulting'
sub_dir = '/map_data/'

In [5]:
folders = glob.glob(base_dir + sub_dir + '/*')
id_nums = [folder.split('/')[-1] for folder in folders]
id_nums

['20167684_07042016', '20157598_12042016', '20167676_01042016']

In [19]:
fol_num = 0

In [20]:
pages = [dir for dir in glob.glob(folders[fol_num] + '/*') if os.path.isdir( dir)]
pages = [p.split('/')[-1] for p in pages]
pages.remove('pages')
pages.remove('JSON')
pages = [int(p) for p in pages]

In [21]:
page_folders = []
for page in sorted(pages):
    page_folder = KML.Folder(
        KML.name('Page ' + str(page+1)),
    )
    kml_files = glob.glob(folders[fol_num] + '/' + str(page) + '/*.kml')
    for kml_file in kml_files:
        with open(kml_file) as f:
            kml = parser.parse(f)
        contents = [
            c for c in list(kml.getroot().Document.iterchildren()) 
            if 'folder' in c.tag.lower()
        ]
        [page_folder.append(c) for c in contents]
    page_folders.append(page_folder)

In [22]:
doc = KML.Document(
    KML.name(id_nums[fol_num].replace('_','.')),
)
[doc.append(pf) for pf in page_folders]

big_kml = KML.kml(
    doc,
)

In [23]:
with open(base_dir + sub_dir + '/' + id_nums[fol_num] + '/' + id_nums[fol_num] + '.kml', 'w') as f:
    f.write(etree.tostring(big_kml, pretty_print=1, encoding='unicode'))